In [ ]:
import pyautogui

print(pyautogui.position())

In [ ]:
import keyboard
import pyautogui
from tkinter.filedialog import *
import datetime

def s_shot():
    myScreenshot = pyautogui.screenshot(region=(55, 523, 675, 370)) # screenshot(region=(x축, y축, 가로길이, 세로길이))
    now = datetime.datetime.now().strftime("%d_%H-%M-%S")
    path = r"c:/data/capture/grand/"+now+"_screenshot.png"
    myScreenshot.save(path)

keyboard.add_hotkey("ctrl+b", lambda: s_shot())

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from sklearn.cluster import KMeans
import datetime
from collections import Counter
import glob

def palette_perc(clt):
    n_pixels = len(clt.labels_)
    counter = Counter(clt.labels_)
    perc = {}
    for i in counter:
        perc[i] = np.round(counter[i]/n_pixels, 2)
    perc = dict(sorted(perc.items()))
    print(perc)

def centroid_histogram(clt):
    numLabels = np.arange(0, len(np.unique(clt.labels_))+1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

def plot_colors(hist, centroids):
    bar = np.zeros((50,300,3), dtype="uint8")
    startX = 0
    for (percent, color) in zip(hist, centroids):
        endX = startX + (percent*300)
        cv2.rectangle(bar, (int(startX),0), (int(endX),50), color.astype("uint8").tolist(), -1)
        startX = endX
    return bar

def image_color_cluster(image_path, k=5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((image.shape[0]*image.shape[1], 3))

    clt = KMeans(n_clusters=k)
    clt.fit(image)

    hist = centroid_histogram(clt)
    bar = plot_colors(hist, clt.cluster_centers_)

    # palette_perc(clt)
    
    plt.figure()
    plt.axis("off")
    plt.imshow(bar)
    plt.show()
    now = datetime.datetime.now().strftime("%d_%H-%M-%S")
    plt.imsave("c:/data/capture/grand/processed/"+now+"_processed.png", bar)

files=list(glob.glob('c:/data/capture/grand/*.png'))

for f in files:
    image = mpimg.imread(f)
    image_color_cluster(f)